<a href="https://colab.research.google.com/github/cras-lab/Finance/blob/main/Efficient_Frontier_N_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

N개 주식에서 최적의 포트폴리오를 구성하는 프로그램

Finance data reader를 설치한다.

In [1]:
pip install -q finance-datareader

필요한 모듈을 임포트 한다.

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import FinanceDataReader as fdr

투자 Universe를 정의한다.

In [132]:
names = ['SSE', 'LGE', 'Naver',  'LGChem', 'KB' ] # Company Name
stocks = ['005930', '066570', '035420', '051910', '105560'  ] #symbol

몇 가지 매개변수를 정의한다.

In [133]:
SCALE_FACTOR = 250  # 연환산 계수
NUM_SIMULATION = 20000  # 시뮬레이션 횟수

종목 시세를 받아올 데이터프레임을 선언한다.

In [134]:
df = pd.DataFrame()

루프를 돌며, 주가를 읽어온다.

In [135]:
for stock in stocks:
    df[stock] = fdr.DataReader(stock, '2019-01-01', '2022-12-31').Close

가독성을 위해 열에 종목 이름을 넣는다.

In [136]:
df.columns = names

일별 수익률을 구한다

In [137]:
daily_ret = df.pct_change()

일별수익률에 연환산 계수를 곱해, 연환산한다.

In [138]:
annual_ret = daily_ret.mean() * SCALE_FACTOR

모든 주식 조합에 대해 서로 공분산을 구한뒤 이 값도 연환산한다.

In [139]:
daily_cov = daily_ret.cov() # 데이터프레임의 공분산 함수 cov() 이용
annual_cov = daily_cov * SCALE_FACTOR # 연환산

각 가중치에 대한 포트폴리오의 수익률, 위험, 가중치, 샤프지수를 저장할 배열 선언

In [140]:
port_return = []  # 포트폴리오 수익률
port_risk = []  # 포트폴리오 위험
port_weights = [] # 포트폴리오 가중치
port_sharpe = []  # 포트폴리오 샤프지수

시뮬레이션 횟수만큼 루프를 돌며, 임의의 가중치에 대해 계산

In [141]:
for i in range(NUM_SIMULATION):
  weights =np.random.random( len(names) ) # 종목만크 임의의 가중치 생성
  weights /= np.sum(weights)  # 가중치의 합은 1이 되도록 조정

  ret = np.dot( weights, annual_ret ) # 현 가중치에 해당하는 포트폴리오 수익률 Rp = SUM( WiRi )
  risk = np.sqrt( np.dot( weights.T, np.dot(annual_cov, weights ) ) ) # 리스크 = SUM(SUM( WiWjCov(i,j)))
  sharpe = ret/risk     # 현 구성에 해당하는 샤프지수 계산
  # 계산한 값들을 배열에 추가
  port_return.append( ret )
  port_risk.append( risk )
  port_sharpe.append( ret / risk )
  port_weights.append( weights )

배열 전체에서 가장 높은 샤프지수를 가진 index를 찾는다.

In [142]:
max_index = port_sharpe.index(max(port_sharpe))

배열 전체에서 가장 낮은 리스크를 가진 index를 찾는다.

In [143]:
min_index = port_risk.index(min(port_risk))

이제 해당 포트폴리오를 도식화 해 본다.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
ax = plt.subplot( )

ax.set_xlabel('Risk')
ax.set_ylabel('Return')
plt.scatter( port_risk, port_return, c=port_sharpe, cmap='viridis', edgecolors='k' ) # k는 검은색을 의미한다.

plt.scatter(x=port_risk[max_index], y=port_return[max_index], c='r', marker='X', s=200)
plt.annotate( f'{names}\n{[f"{x:.2f}" for x in port_weights[max_index]]}\n MAX SHARPE = {port_sharpe[max_index]:.2f}', (port_risk[max_index], port_return[max_index]), xytext=(-140,25), textcoords='offset points')

plt.scatter(x=port_risk[min_index], y=port_return[min_index], c='r', marker='X', s=200)
plt.annotate( f'{names}\n{[f"{x:.2f}" for x in port_weights[min_index]]}\nMIN RISK /w SHARPE = {port_sharpe[min_index]:.2f}', (port_risk[min_index], port_return[min_index]), xytext=(-180,-10), textcoords='offset points')

colorbar = plt.colorbar()
colorbar.set_label("Sharpe Ratio")
